In [2]:
from pinscrape import scraper, Pinterest

def using_search_engine(keyword, output_folder, images_to_download, proxies={}, number_of_workers=10):

    details = scraper.scrape(keyword, output_folder, proxies, number_of_workers, images_to_download)
    if details["isDownloaded"]:
        print("\nDownloading completed !!")
        print(f"\nTotal urls found: {len(details['extracted_urls'])}")
        print(f"\nTotal images downloaded (including duplicate images): {len(details['urls_list'])}")
        print(details)
    else:
        print("\nNothing to download !!", details)



In [3]:

def using_pinterest_apis(keyword, output_folder, images_to_download, proxies={}, number_of_workers=20):

    p = Pinterest(proxies=proxies) 
    images_url = p.search(keyword, images_to_download)
    p.download(url_list=images_url, number_of_workers=number_of_workers, output_folder=output_folder)


In [4]:
%pip install pinscrape


Note: you may need to restart the kernel to use updated packages.


In [6]:
season_tuple = (
    'Spring 2020', 'Summer 2020', 'Fall 2020', 'Autumn 2020', 'Winter 2020',
    'Spring 2021', 'Summer 2021', 'Fall 2021', 'Autumn 2021', 'Winter 2021',
    'Spring 2022', 'Summer 2022', 'Fall 2022', 'Autumn 2022', 'Winter 2022',
    'Spring 2023', 'Summer 2023', 'Fall 2023', 'Autumn 2023', 'Winter 2023',
    'Spring 2024', 'Summer 2024', 'Fall 2024', 'Autumn 2024', 'Winter 2024',
)

# Number of images to download
images_to_download = 150

# Loop through the months tuple
for month in season_tuple:
    keyword = f"dresses women casual {month}"
    output_folder = f"images\{month}"
    using_pinterest_apis(keyword, output_folder, images_to_download)


In [12]:
import os

# Fix for OpenMP library conflict
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["OMP_NUM_THREADS"] = "1"  # Limit OpenMP threads

import torch
import pandas as pd
from pathlib import Path
from hashlib import md5
from collections import defaultdict
from ultralytics import YOLO

def process_folder_with_yolo(folder_path, output_path_1, output_path_2, model):
    """
    Processes a folder to filter out non-portrait/selfie images using YOLOv5.
    Args:
        folder_path (str): Path to the folder containing images.
        output_path_1 (str): Path to save images we need.
        output_path_2 (str): Path to save images we don't need.
        model: Pre-trained YOLOv5 model.
    """
    # Create output folders if they don't exist
    os.makedirs(output_path_1, exist_ok=True)
    os.makedirs(output_path_2, exist_ok=True)

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an image
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Run YOLOv5 model inference
            results = model.predict(source=file_path, device='cuda')
            detections = results[0].boxes.data.cpu().numpy()  # Get detections as a NumPy array

            # Filter detections for 'person' class (class index 0 is typically 'person')
            people = [det for det in detections if int(det[-1]) == 0]

            # Save to appropriate folder based on detection
            if len(people) > 0:
                output_file_path = os.path.join(output_path_1, file_name)
            else:
                output_file_path = os.path.join(output_path_2, file_name)

            # Save the image
            os.rename(file_path, output_file_path)
    """
    Processes a folder to filter out non-portrait/selfie images using YOLOv5.
    Args:
        folder_path (str): Path to the folder containing images.
        output_path_1 (str): Path to save images we need.
        output_path_2 (str): Path to save images we don't need.
        model: Pre-trained YOLOv5 model.
    """
    # Create output folders if they don't exist
    os.makedirs(output_path_1, exist_ok=True)
    os.makedirs(output_path_2, exist_ok=True)

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an image
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Run YOLOv5 model inference
            results = model(file_path)
            detections = results.pandas().xyxy[0]  # Get detections as a DataFrame

            # Filter detections for 'person' class
            people = detections[detections['name'] == 'person']

            # Save to appropriate folder based on detection
            if len(people) > 0:
                output_file_path = os.path.join(output_path_1, file_name)
            else:
                output_file_path = os.path.join(output_path_2, file_name)

            # Save the image
            os.rename(file_path, output_file_path)

def remove_seasonal_duplicates(base_path, season_tuple):
    """
    Removes duplicate images across the same season by comparing file hashes.
    Args:
        base_path (str): Path to the base directory containing season folders.
        season_tuple (tuple): List of all subfolder names.
    """
    duplicates_count = 0
    season_groups = defaultdict(list)

    # Group folders by season (e.g., 'Fall', 'Spring')
    for folder_name in season_tuple:
        season = folder_name.split()[0]  # Extract the season (e.g., 'Spring', 'Fall')
        season_groups[season].append(folder_name)

    for season, folders in season_groups.items():
        print(f"Checking for duplicates within season: {season}")
        image_hashes = {}

        for folder_name in folders:
            folder_path = os.path.join(base_path, folder_name)

            if os.path.exists(folder_path):
                for file_name in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file_name)

                    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        with open(file_path, 'rb') as f:
                            file_hash = md5(f.read()).hexdigest()

                        if file_hash in image_hashes:
                            print(f"Duplicate found: {file_path} matches {image_hashes[file_hash]}. Removing.")
                            os.remove(file_path)
                            duplicates_count += 1
                            # Log duplicate removal
                            print(f"Duplicate count updated: {duplicates_count}")
                        else:
                            image_hashes[file_hash] = file_path

    return duplicates_count

def process_all_folders(base_path, season_tuple, images_1_path, images_2_path, model):
    """
    Processes all folders in season_tuple.
    Args:
        base_path (str): Path to the base directory containing folders.
        season_tuple (tuple): List of all subfolder names.
        images_1_path (str): Path to the output directory for images_1.
        images_2_path (str): Path to the output directory for images_2.
        model: Pre-trained YOLOv5 model.
    """
    for folder_name in season_tuple:
        folder_path = os.path.join(base_path, folder_name)

        if os.path.exists(folder_path):
            print(f"Processing folder: {folder_name}")
            output_path_1 = os.path.join(images_1_path, folder_name)
            output_path_2 = os.path.join(images_2_path, folder_name)
            process_folder_with_yolo(folder_path, output_path_1, output_path_2, model)

def calculate_distribution(base_path, season_tuple, images_1_path, images_2_path, duplicates_count):
    """
    Calculates the distribution of images across all folders.
    Args:
        base_path (str): Path to the base directory containing folders.
        season_tuple (tuple): List of all subfolder names.
        images_1_path (str): Path to the output directory for images_1.
        images_2_path (str): Path to the output directory for images_2.
        duplicates_count (int): Number of duplicate images removed.
    """
    distribution = []

    for folder_name in season_tuple:
        images_1_folder = os.path.join(images_1_path, folder_name)
        images_2_folder = os.path.join(images_2_path, folder_name)

        images_1_count = len(os.listdir(images_1_folder)) if os.path.exists(images_1_folder) else 0
        images_2_count = len(os.listdir(images_2_folder)) if os.path.exists(images_2_folder) else 0

        distribution.append({
            "Folder": folder_name,
            "Images_1": images_1_count,
            "Images_2": images_2_count,
            "Total": images_1_count + images_2_count
        })

    total_images_1 = sum(d["Images_1"] for d in distribution)
    total_images_2 = sum(d["Images_2"] for d in distribution)

    distribution.append({
        "Folder": "All Folders",
        "Images_1": total_images_1,
        "Images_2": total_images_2,
        "Deleted Duplicates": duplicates_count,
        "Total": total_images_1 + total_images_2
    })

    df = pd.DataFrame(distribution)
    return df


In [ ]:

if __name__ == "__main__":
    # Path to the dataset base directory
    base_directory = "images"

    # Path to images_1 and images_2 directories
    images_1_directory = "images1"
    images_2_directory = "images2"

    # Tuple containing all subfolder names
    season_tuple = (
        'Spring 2020', 'Summer 2020', 'Fall 2020', 'Autumn 2020', 'Winter 2020',
        'Spring 2021', 'Summer 2021', 'Fall 2021', 'Autumn 2021', 'Winter 2021',
        'Spring 2022', 'Summer 2022', 'Fall 2022', 'Autumn 2022', 'Winter 2022',
        'Spring 2023', 'Summer 2023', 'Fall 2023', 'Autumn 2023', 'Winter 2023',
        'Spring 2024', 'Summer 2024', 'Fall 2024', 'Autumn 2024', 'Winter 2024',
    )

    # Remove duplicates within each season
    duplicates_count = remove_seasonal_duplicates(base_directory, season_tuple)


In [13]:
if __name__ == "__main__":
    
    # Load YOLOv5 model using Ultralytics package
    model = YOLO('yolov5s.pt')

    # Process all folders
    process_all_folders(base_directory, season_tuple, images_1_directory, images_2_directory, model)
    print("Filtering complete!")

    # Calculate and display distribution
    distribution_df = calculate_distribution(base_directory, season_tuple, images_1_directory, images_2_directory, duplicates_count)
    print(distribution_df)
    distribution_df.to_csv("distribution.csv", index=False)


PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Processing folder: Spring 2020

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\00aa407679c5cb9b36951b1d0c2349bc.jpg: 640x448 1 person, 12 cars, 1 truck, 1 handbag, 30.2ms
Speed: 3.0ms preprocess, 30.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\015d9ae3942d1daa597e6721525a7d87.jpg: 640x512 1 person, 137.6ms
Speed: 1.5ms preprocess, 137.6ms inference, 6.1ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\0214ee050985ae51bb02f0a41d630d5e.jpg: 640x352 1 pe

c:\Users\djsma\anaconda3\envs\xtuner-env\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Speed: 1.5ms preprocess, 11.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\44e2ab4710cdfabd7d9c9dd59f872c26.jpg: 640x640 1 person, 15.8ms
Speed: 3.0ms preprocess, 15.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\4561df0dc0eab751f87b6b785ec39ab3.jpg: 640x352 4 persons, 2 cars, 1 handbag, 14.7ms
Speed: 2.5ms preprocess, 14.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images\Spring 2020\4653fdded29f213e98b9c3c002fbed19.jpg: 640x448 1 person, 11.0ms
Speed: 2.5ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 c:\Users\djsma\OneDrive\Documents\GitHub\VogueFusion_Pulse\fashion_trends\images